In [1]:
import ee
#ee.Authenticate()
ee.Initialize()
#print(ee.Image("NASA/NASADEM_HGT/001").get("title").getInfo())
import geemap
import folium

### Long Beach Ship Detection

In [5]:
# Create a default map
Map = geemap.Map()

# Load an image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Create an NDWI image, define visualization parameters and display.
ndwi = image.normalizedDifference(['B3', 'B5'])
ndwiViz = {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}
Map.setCenter(-122.1899, 37.5010, zoom=10)  # SLong Beach
Map.addLayer(ndwi, ndwiViz, 'NDWI', False)

# Display the map
Map


Map(center=[37.501, -122.1899], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…